<a href="https://colab.research.google.com/github/KrungSalad/Google_Colab/blob/master/Assignment_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score,accuracy_score
# load dataset into Pandas DataFrame
df = pd.read_csv('cumulative.csv')
df.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


Data Cleaning & Max-Min Normalize

In [2]:
df.dropna(axis = 1, how = 'all', inplace = True)
df.fillna(method='ffill' , inplace = True )
df.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,Kepler-227 c,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,Kepler-227 c,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
q = df.drop('rowid', axis = 1)
q = q.drop('kepid', axis = 1)
q = q.drop('kepoi_name', axis = 1)
q = q.drop('koi_disposition', axis = 1)
q = q.drop('koi_pdisposition', axis = 1)
q = q.drop('koi_tce_delivname', axis = 1)
q = q.drop('kepler_name', axis = 1)
q.head()

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,93.59,29.45,-16.65,35.8,1.0,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,9.11,2.87,-1.62,25.8,2.0,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,39.30,31.04,-10.49,76.3,1.0,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,891.96,668.95,-230.35,505.6,1.0,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,926.16,874.33,-314.24,40.9,1.0,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
from sklearn import preprocessing
x = q.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_normalized = pd.DataFrame(x_scaled, columns=q.columns)
X_normalized.head()

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1.000,0.0,0.0,0.0,0.0,0.000071,0.000161,0.999839,0.036999,0.003781,0.996219,0.001448,0.003718,0.997539,0.020980,0.004054,0.995946,0.000400,0.000050,0.999950,0.000011,0.000012,0.999998,0.052452,8.548941e-06,8.141808e-06,0.999997,0.003954,0.000000,0.211107,0.119822,0.954030,0.831296,0.043478,0.920464,0.003560,0.003173,0.999475,0.552473,0.733837,0.642863
1,0.969,0.0,0.0,0.0,0.0,0.000417,0.001437,0.998563,0.031063,0.006171,0.993829,0.005813,0.000690,0.992532,0.032169,0.005743,0.994257,0.000568,0.000091,0.999909,0.000014,0.000015,0.999998,0.028548,8.321493e-07,7.934462e-07,1.000000,0.002849,0.142857,0.211107,0.119822,0.954030,0.831296,0.043478,0.920464,0.003560,0.003173,0.999475,0.552473,0.733837,0.642863
2,0.000,0.0,1.0,0.0,0.0,0.000151,0.000087,0.999913,0.040928,0.001006,0.998994,0.009613,0.059925,0.998702,0.012494,0.001688,0.998312,0.007025,0.000440,0.999560,0.000072,0.000181,0.999983,0.041866,3.589843e-06,8.581383e-06,0.999998,0.008427,0.000000,0.241179,0.233728,0.900114,0.845778,0.029891,0.854184,0.003303,0.007041,0.999328,0.784346,0.733360,0.649689
3,0.000,0.0,1.0,0.0,0.0,0.000012,0.000002,0.999998,0.036828,0.000187,0.999813,0.012658,0.001344,0.998449,0.017001,0.000266,0.999734,0.005241,0.000033,0.999967,0.000167,0.000393,0.999963,0.093566,8.147573e-05,1.849393e-04,0.999959,0.055838,0.000000,0.237552,0.232249,0.901249,0.849539,0.036005,0.860812,0.002968,0.006074,0.999423,0.259826,0.742947,0.662039
4,1.000,0.0,0.0,0.0,0.0,0.000018,0.000022,0.999978,0.037781,0.001971,0.998029,0.006954,0.002747,0.991942,0.011571,0.002079,0.997921,0.000391,0.000043,0.999957,0.000013,0.000041,0.999995,0.094318,8.459972e-05,2.417191e-04,0.999944,0.004517,0.000000,0.254628,0.250000,0.880250,0.825842,0.047554,0.826015,0.004077,0.010093,0.998855,0.407085,0.739203,0.655289


PCA

In [5]:

variables = ['koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1',
       'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_steff_err1', 'koi_steff_err2',
       'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad',
       'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'koi_kepmag']
x = X_normalized.loc[:, variables].values
y = X_normalized.loc[:,['koi_score']].values

from sklearn.decomposition import PCA
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'
             , 'principal component 5'])

finalDf = pd.concat([principalDf, X_normalized[['koi_score']]], axis = 1)
display(finalDf.head())
display(pca.explained_variance_ratio_)

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,koi_score
0,-0.251969,-0.178300,-0.265320,0.069888,0.234317,1.000
1,-0.258561,-0.178332,-0.268659,0.070113,0.234014,0.969
2,0.413915,-0.602619,0.362099,0.047900,0.312117,0.000
3,0.374239,-0.630090,0.344860,0.172129,0.208136,0.000
4,-0.257432,-0.180415,-0.256864,0.101715,0.213785,1.000


array([0.29693626, 0.23759124, 0.15381005, 0.07627084, 0.0645923 ])

Plot

In [0]:
#sns.pairplot(df[['koi_period', 'koi_duration','koi_depth', 'koi_prad', 'koi_teq','koi_score']], kind='reg');

Data

In [0]:
from sklearn.model_selection import train_test_split
# PCA
X = finalDf[['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5']]
y = finalDf['koi_score'].astype('int')

#X = df[['koi_period', 'koi_duration','koi_depth', 'koi_prad', 'koi_teq']]
#y = df['koi_score'].astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 2)


In [8]:
display(X_train)
display(y_train)

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5
377,-0.234022,-0.159887,-0.242679,-0.024383,-0.199480
2661,-0.281577,-0.184501,-0.256785,0.106944,-0.172607
9354,-0.219868,-0.151137,-0.229783,-0.054885,0.085858
7494,0.418699,-0.647473,0.430962,0.090062,0.222372
2207,-0.246394,-0.159122,-0.225524,-0.007197,-0.391896
...,...,...,...,...,...
1099,0.409830,-0.597602,0.388057,0.034327,0.170981
2514,-0.228487,-0.156485,-0.236506,-0.027335,-0.010945
6637,-0.219970,-0.147428,-0.210541,-0.061008,-0.055900
2575,-0.260425,-0.169917,-0.244108,0.035633,-0.034055


377     0
2661    0
9354    1
7494    0
2207    0
       ..
1099    0
2514    1
6637    0
2575    1
7336    0
Name: koi_score, Length: 7651, dtype: int64

In [9]:
display(X_test)
display(y_test)

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5
9276,-0.276191,-0.186599,-0.262336,0.149384,0.244468
6321,-0.576295,0.394113,0.509540,-0.019071,0.282273
2006,-0.252550,-0.173502,-0.250241,0.068380,0.055001
2200,-0.242863,-0.162519,-0.242125,0.004651,-0.068587
5923,-0.199511,-0.131899,-0.207691,-0.125934,-0.165722
...,...,...,...,...,...
1939,-0.213463,-0.148401,-0.229298,-0.084994,-0.102412
5305,0.422659,-0.584104,0.383977,-0.053188,-0.345065
8643,0.013580,0.967683,0.331150,-0.601968,0.249540
2602,-0.282711,-0.188719,-0.266740,0.127665,-0.117506


9276    0
6321    0
2006    1
2200    0
5923    0
       ..
1939    1
5305    0
8643    0
2602    1
48      1
Name: koi_score, Length: 1913, dtype: int64

SVC

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
#accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
#accuracies.mean()

Accuracy ของ Data ที่นำไป Train Model SVC โดยใช้ K-Fold

In [11]:
from sklearn import model_selection 
from sklearn.model_selection import cross_val_score 

kfold = model_selection.KFold(n_splits=10, random_state=2)
clf = SVC(kernel = 'rbf', random_state = 0)
results = model_selection.cross_val_score (clf, X_train, y_train, cv=kfold, scoring= 'accuracy')
print("10-fold accuracy: %.3f" % (results.mean()))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


10-fold accuracy: 0.802


ความถูกต้องของ Data ระหว่าง y_test , y_pred สำหรับ Model SVC

In [12]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88      1514
           1       0.00      0.00      0.00       399

    accuracy                           0.79      1913
   macro avg       0.40      0.50      0.44      1913
weighted avg       0.63      0.79      0.70      1913

[[1514    0]
 [ 399    0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LogisticRegression

In [0]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#accuracy = model.score(X_test, y_test)
#print(accuracy)

Accuracy ของ Data ที่นำไป Train Model LogisticRegression โดยใช้ K-Fold

In [14]:
from sklearn import model_selection 
from sklearn.model_selection import cross_val_score 

kfold = model_selection.KFold(n_splits=10, random_state=2)
clf = LogisticRegression()
results = model_selection.cross_val_score (clf, X_train, y_train, cv=kfold, scoring= 'accuracy')
print("10-fold accuracy: %.3f" % (results.mean()))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


10-fold accuracy: 0.802


ความถูกต้องของ Data ระหว่าง y_test , y_pred สำหรับ Model LogisticRegression

In [15]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88      1514
           1       1.00      0.01      0.01       399

    accuracy                           0.79      1913
   macro avg       0.90      0.50      0.45      1913
weighted avg       0.84      0.79      0.70      1913

[[1514    0]
 [ 397    2]]


In [16]:
display("y_test : ",y_test)
display("y_pred : ",y_pred)

'y_test : '

9276    0
6321    0
2006    1
2200    0
5923    0
       ..
1939    1
5305    0
8643    0
2602    1
48      1
Name: koi_score, Length: 1913, dtype: int64

'y_pred : '

array([0, 0, 0, ..., 0, 0, 0])

LinearDiscriminantAnalysis (LDA)

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
Model=LinearDiscriminantAnalysis()
Model.fit(X_train,y_train)
y_pred=Model.predict(X_test)
#print('accuracy is ',accuracy_score(y_pred,y_test))

Accuracy ของ Data ที่นำไป Train Model LDA โดยใช้ K-Fold

In [18]:
from sklearn import model_selection 
from sklearn.model_selection import cross_val_score 

kfold = model_selection.KFold(n_splits=10, random_state=2)
clf = LinearDiscriminantAnalysis()
results = model_selection.cross_val_score (clf, X_train, y_train, cv=kfold, scoring= 'accuracy')
print("10-fold accuracy: %.3f" % (results.mean()))

10-fold accuracy: 0.802


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


ความถูกต้องของ Data ระหว่าง y_test , y_pred สำหรับ Model LDA

In [19]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88      1514
           1       0.00      0.00      0.00       399

    accuracy                           0.79      1913
   macro avg       0.40      0.50      0.44      1913
weighted avg       0.63      0.79      0.70      1913

[[1514    0]
 [ 399    0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
